# Constructing CNNs with Keras

In this lab, we will look at constructing convolutional networks with Keras. Keras drastically simplifies the process of constructing a network by using predefined network models and layers. 

After installing Keras via `pip install keras` restart your kernel. We will start by building a multilayer perceptron, and then move on to CNNs. Our first data set will be our old friend the MNIST dataset. Lets build a perceptron with a perceptron with one deep layer:

<img width = 500 src = "http://i64.tinypic.com/e980nm.png">

We will use this to classify the MNIST dataset. First, we load in MNIST, normalize the X pixel intensities to be between 0 and 1 (this works better with our activation functions) and then encode the labels $Y$ as one-hot encoded categorical variables. We use the `keras.utils` class to do the one-hot encoding, but you could use `pandas.get_dummies` as well.

This tutorial follows closely the excellent tutorials https://nextjournal.com/gkoehler/digit-recognition-with-keras and https://colab.research.google.com/github/csc-training/intro-to-dl/blob/master/day1/keras-mnist-mlp.ipynb#scrollTo=-fGvYAbhVADs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist, fashion_mnist
from keras.utils import np_utils

## MNIST:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

NUM_LABELS = 10

## Normalize training data to be between 0 and 1, we have to typecast it as a float to do so.
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')
X_train /= 255
X_test /= 255

X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

# one-hot encoding:
Y_train = np_utils.to_categorical(y_train, NUM_LABELS)
Y_test = np_utils.to_categorical(y_test, NUM_LABELS)

print()
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('x_train:', x_train.shape)
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('Y_train:', Y_train.shape)

To build a sequential model in Keras we import the `sequential` class from `keras.model` and then the types of layers we will use from `keras.labels`. In this case we will be using __dense__ layer and __activation__ layers. 

We then define a new model with `model = Sequential()`. We add layers in sequence with `model.add(layer)` and Keras takes care of the connects for us:


<div class="alert alert-block alert-info">
__Dense__ implements the operation: __output = activation(dot(input, kernel) + bias)__ where activation is the element-wise activation function passed as the activation argument, kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (only applicable if use_bias is True).

[Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)
</div>

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(256, input_shape=(784,),activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Activation('softmax'))

That's it, we've constructed out model. To compile it we use `model.compile`. We will use the Adam optimizer, which automatically sets a different learning schedule for each weight. For more information about Adam, see [Jason Brownlee's excelent post](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) or [the original paper of Kingma and Ba](https://arxiv.org/abs/1412.6980). 

To train the model, use

`model.fit(X_train, Y_train,
          batch_size=, epochs=20,
          verbose=2,
          validation_data=(X_test, Y_test))`

* `X_train` is the whole set of training data.
* `Y_train` is the whole set of label data.
* `batch_size` is size of each training minibatch. Remember that 1 is __stochastic gradient decent__ while 60000 (the size of the whole data set) would be __gradient decent__. 
* `verbose` sets how much information to output during fitting. 0 = silent, 1 = progress bar, 2 = one line per epoch.
* `validation_data=()` specifies data to validate on after each training epoch. 

We will save the output of the training in a variable called `history` for later viewing. 

https://keras.io/models/model/

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model and saving metrics in history
history = model.fit(X_train, Y_train,
          batch_size=128, epochs=20,
          verbose=1,
          validation_data=(X_test, Y_test))

#### Saving weights with Keras

Unlike pure tensorflow, Keras will track the tensorflow session by default, including keeping the model alive in a session for us. To save out the weights, we use `model.save(FILE_NAME)`. The weights can be recovered by using `model.load_weights`. Be warned: you have to build a model with the same architecture first and then load the weights into it. 

In [ ]:
# saving the model
SAVE_DIR = "./"
MODEL_NAME = 'keras_mnist.h5'
model_path = os.path.join(SAVE_DIR, MODEL_NAME)

model.save(model_path)
print('Saved trained model at %s ' % model_path)

#### Visualizing Training

Keras also saves the models training history. History store the training accuracy, the validation accuracy and the training loss and the validation loss. Below, we plot them against the epoch number

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()

fig

## Question:

By increasing the layer size, adding more layers, or training time, try to get the validation error above 97%. You may want to add a dropout layer to speed up performance. To add a dropout layer, just put

    model.add(Dropout(P)) 

after a dense layer, where `P` is the proportion of connections to turn off.

## Convolutional Neural Networks

Lets try out our perceptron network on something a little more complicated. The fashion MNIST dataset has a very similar structure to MNIST accept that instead of simple hand written digits it contains $28\times 28$ images of items of clothing:

<img src="http://i64.tinypic.com/10711u0.png">

We load it as below and process it with the same code as before. Take a moment to poke around the dataset before processing it, the labels are

Label|Description|Label|Description
--- | --- |--- | ---
0|T-shirt/top|5|Sandal
1|Trouser|6|Shirt
2|Pullover|7|Sneaker
3|Dress|8|Bag
4|Coat|9|Ankle boot

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
plt.imshow(x_train[0])

Processing the data:

In [ ]:
NUM_LABELS = 10

## Normalize training data to be between 0 and 1, we have to typecast it as a float to do so.
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')
X_train /= 255
X_test /= 255

X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

# one-hot encoding:
Y_train = np_utils.to_categorical(y_train, NUM_LABELS)
Y_test = np_utils.to_categorical(y_test, NUM_LABELS)

print()
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('x_train:', x_train.shape)
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('Y_train:', Y_train.shape)

Now, try running the perceptron network about and note the validation error, you may need to increase the number of epochs to get reasonable results. 

#### Convolutional Neural Networks

My error on the MLP never capped 82%, even with 100 epochs. Can we do better with a CNN? Recall that CNN's are comprised of stacks of convolution layers, activation layers, pooling layers and finally a flattening layer:

<table bgcolor="#fafafa"><tr>
    <td>__Convolution Layer__</td><td><img width=400 src="http://i66.tinypic.com/260r9sw.png">
    </tr>
    <td>__Pooling Layer__</td><td><img width=300 src="http://i66.tinypic.com/19nxc1.png">
    </tr><tr>
    <td>__Flattening Layer__</td><td><img width=100 src="http://i65.tinypic.com/oau6vr.png">
    </tr></table>
    
A convolution layer is defined with    
    
    `Conv2D(nb_filters, kernel_size,
                 padding='valid',
                 input_shape=input_shape,
                 activation='relu')`
                 
Where

* `nb_filters` number of convolution filters.
* `kernel_size` size of each filter, say [5,5] for a $5\times 5$ filter.
* `padding` When we convolve, we tend to lower the image size. We can choose to pad the image back to its original size or not. 
* `input_shape` shape of the inputed training data, only required for the first layer. 
* `activation` the activation layer following the convolution layer. 

For a pooling layer we only specify the pool size:

* `MaxPooling2D(pool_size=pool_size)` where `pool_size = [2,2]` down-samples by 2 in each direction. 

After we down-sample enough, we flatten and feed the network into a dense layer to do the fitting. The final architecture looks like the cartoon from class:

<img width= 700 src="http://i67.tinypic.com/21eca3q.png">

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D
from keras.layers.convolutional import Conv2D 

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

## We renormalize the training data since we do not need to flatten it
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')
X_train /= 255
X_test /= 255

## We have to add an extra dimension to allow for the multiple images we will be creating
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

Y_train = np_utils.to_categorical(y_train, NUM_LABELS)
Y_test = np_utils.to_categorical(y_test, NUM_LABELS)

Now, lets create our model. Our convolution layers will have $3\times 3$ filters followed by downsampling.

In [ ]:
model = Sequential()

model.add(Conv2D(32, (7,7),
                 padding='valid',
                 input_shape=(28, 28,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=NUM_LABELS, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(model.summary())

Lets fit, and plot the results. Each epoch will take 10-100 s depending on your processor. 

In [ ]:
epochs = 5

history = model.fit(X_train, 
                    Y_train, 
                    epochs=epochs, 
                    batch_size=128,
                    verbose=1,
                    validation_data=(X_test, Y_test))

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()

fig

## Understanding the model

There are two things we would like to understand about a CNN: __what is it doing__ and __what is it not doing__? To answer the first question let open up the box a bit and see what the first few convolution kernels look like. 

The `model.layers[]` array gives a list of handlers for the model layers in the order given by summary. Note that you can also name your layers and call them that way. We then use `layer.get_weights()` to return the convolution and bias weights for each of the 32 kernel layers. 

In [ ]:
weights, biases = model.layers[0].get_weights()
print(weights.shape)

plt.imshow(weights[:,:,0,9],cmap="Greys")

#### Exercise:

Display all of the kernels in a grid. In addition, normalize the color scheme so that each image uses the same scheme. 

On the otherhand, its important to know what we're getting wrong. Lets construct the confusion matrix to discern which images the network has the hardest time classifying. 

Label|Description|Label|Description
--- | --- |--- | ---
0|T-shirt/top|5|Sandal
1|Trouser|6|Shirt
2|Pullover|7|Sneaker
3|Dress|8|Bag
4|Coat|9|Ankle boot

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_hat = np.argmax(model.predict(X_test),axis=1)
conf_mx = confusion_matrix(y_test, y_hat)

## Remove diagonal for better viewing
row_sum = conf_mx.sum(axis=1, keepdims=True)
nconf_mx = conf_mx/row_sum
np.fill_diagonal(nconf_mx,0)

labels=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

sns.heatmap(nconf_mx, xticklabels=labels, yticklabels=labels)

## Question:

Find and display examples of misclassified items of clothing. What could you do to classify them more accurately?

## Question: Model Tuning

Modify the CNN model above to improve the classification accuracy, or experiment with the effects of different parameters. If you are interested in the state-of-the-art performance on permutation invariant MNIST, see the papers here 

http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

and at Yann LeCun's webpage

http://yann.lecun.com/exdb/mnist/

Be sure to consult the the Keras documentation at https://keras.io/.

## Question: Data Augmentation

The best way to increase the efficiency of a model is to start with more data. Create a function that takes the Fashion MNIST dataset and flips every image horizontally, effectively doubling your dataset size. Does this improve your model performance?